# *Detection* Model - ResNet

In [1]:
!pip install adversarial-robustness-toolbox torch matplotlib numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 29.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.8 MB/s eta 0:00:00:00:0100:01
  Attempting unins

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset
import torchvision.models as models
from torchvision.models import resnet18

from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
import numpy as np

import os

In [3]:
baseline_model_resnet = models.resnet18(weights=None) # Start with torchvision's ResNet-18

# Adapt ResNet-18 for CIFAR-10 (32x32 images)
baseline_model_resnet.conv1 = nn.Conv2d(
    3, 64, kernel_size=3, stride=1, padding=1, bias=False
)
baseline_model_resnet.maxpool = nn.Identity() # Remove initial max pooling

# Adapt final layer for 10 CIFAR classes
num_ftrs = baseline_model_resnet.fc.in_features
baseline_model_resnet.fc = nn.Linear(num_ftrs, 10)

# baseline_model_resnet = baseline_model_resnet.to(device)

In [11]:
# defining training training hyperparameters

EPOCHS_DETECTION = 20
BATCH_SIZE = 128
LEARNING_RATE = 0.001
EPSILON = 8/255 # pertuberation size for PGD attack

# checking for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

baseline_model_resnet = baseline_model_resnet.to(device)

Using device: cuda


In [5]:
# CIFAR-10 preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

print("CIFAR-10 training dataset loaded.")

100%|██████████| 170M/170M [00:04<00:00, 40.4MB/s] 


CIFAR-10 training dataset loaded.


In [6]:
# helper function for attack generation

def generate_adversarial_data(classifier, x_data, y_data, attack_method):
    print(f"Generating adversarial examples using {attack_method.__class__.__name__}...")
    x_adv = attack_method.generate(x = x_data, y = y_data)
    return x_adv

In [ ]:
import os

print("Folders in /kaggle/input:")
print(os.listdir("/kaggle/input"))

In [7]:
# training phase - detection model

# need to upload the "baseline_model_resnet.pth" file to the
# working directory `/kaggle/working` before running this cell


baseline_load_path = '/kaggle/input/temp_baseline_resnet_cifar.pth'
baseline_model_resnet.load_state_dict(torch.load(baseline_load_path))
baseline_model_resnet.eval()
print("Baseline model architecture defined using torchvision and weights loaded.")

baseline_classifier_art = PyTorchClassifier(
    model=baseline_model_resnet,
    loss=nn.CrossEntropyLoss(),
    input_shape=(3, 32, 32),
    nb_classes=10, 
    device_type=device.type 
)

pgd_attack = ProjectedGradientDescent(baseline_classifier_art, eps=EPSILON)

print("CIFAR-10 baseline model loaded and PGD attack configured.")

Baseline model architecture defined using torchvision and weights loaded.
CIFAR-10 baseline model loaded and PGD attack configured.


In [8]:
# generating training dataset for detection

# using the test dataset to create the adversarial examples

x_train_np = train_dataset.data.transpose(0, 3, 1, 2).astype(np.float32) / 255.0
y_train_np = np.array(train_dataset.targets)


x_adv_np = generate_adversarial_data(baseline_classifier_art, x_train_np, y_train_np, pgd_attack)

x_detection_np = np.concatenate((x_train_np, x_adv_np), axis=0)
y_detection_np = np.concatenate((np.zeros(len(x_train_np)), np.ones(len(x_adv_np))), axis=0)

x_detection_tensor = torch.from_numpy(x_detection_np).float()
y_detection_tensor = torch.from_numpy(y_detection_np).long()

detection_dataset = TensorDataset(x_detection_tensor, y_detection_tensor)
detection_loader = DataLoader(detection_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"Detection dataset generated. Size: {len(detection_dataset)}")
print(f"Shape of detection images: {x_detection_tensor.shape}")
print(f"Shape of detection labels: {y_detection_tensor.shape}")

Generating adversarial examples using ProjectedGradientDescent...


PGD - Batches:   0%|          | 0/1563 [00:00<?, ?it/s]

Detection dataset generated. Size: 100000
Shape of detection images: torch.Size([100000, 3, 32, 32])
Shape of detection labels: torch.Size([100000])


In [9]:
def ResNet18(num_classes=2):
    model = resnet18(weights=None)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

# Training function used for the detector
def train_model(model, train_loader, optimizer, loss_fn, epochs, description="", scheduler=None):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        if scheduler:
            scheduler.step()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = 100.0 * correct / total

        print(f"[{description}] Epoch [{epoch+1}/{epochs}] - Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.2f}%")

    print("Training complete.")

In [12]:
detection_model_resnet = ResNet18(num_classes = 2).to(device) 

detector_optimizer = optim.Adam(detection_model_resnet.parameters(), lr = LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss() 



print("--- Training ResNet Detection Model ---")
train_model(
    model=detection_model_resnet,
    train_loader=detection_loader, 
    optimizer=detector_optimizer,
    loss_fn=loss_fn,
    epochs=EPOCHS_DETECTION, 
    description="CIFAR-10 ResNet Detector",
    scheduler=None 
)
print("--- Detection Model Training Finished ---")


model_save_dir = 'models'
os.makedirs(model_save_dir, exist_ok=True) 
model_save_path = os.path.join(model_save_dir, "detection_model_resnet_cifar.pth")

torch.save(detection_model_resnet.state_dict(), model_save_path)
print(f"CIFAR-10 ResNet Detection model saved to Kaggle at: {model_save_path}")

--- Training ResNet Detection Model ---
[CIFAR-10 ResNet Detector] Epoch [1/20] - Loss: 0.2234 | Accuracy: 87.35%
[CIFAR-10 ResNet Detector] Epoch [2/20] - Loss: 0.0336 | Accuracy: 98.87%
[CIFAR-10 ResNet Detector] Epoch [3/20] - Loss: 0.0215 | Accuracy: 99.27%
[CIFAR-10 ResNet Detector] Epoch [4/20] - Loss: 0.0193 | Accuracy: 99.33%
[CIFAR-10 ResNet Detector] Epoch [5/20] - Loss: 0.0118 | Accuracy: 99.59%
[CIFAR-10 ResNet Detector] Epoch [6/20] - Loss: 0.0153 | Accuracy: 99.51%
[CIFAR-10 ResNet Detector] Epoch [7/20] - Loss: 0.0254 | Accuracy: 99.16%
[CIFAR-10 ResNet Detector] Epoch [8/20] - Loss: 0.0082 | Accuracy: 99.72%
[CIFAR-10 ResNet Detector] Epoch [9/20] - Loss: 0.0085 | Accuracy: 99.70%
[CIFAR-10 ResNet Detector] Epoch [10/20] - Loss: 0.0110 | Accuracy: 99.63%
[CIFAR-10 ResNet Detector] Epoch [11/20] - Loss: 0.0084 | Accuracy: 99.73%
[CIFAR-10 ResNet Detector] Epoch [12/20] - Loss: 0.0092 | Accuracy: 99.67%
[CIFAR-10 ResNet Detector] Epoch [13/20] - Loss: 0.0059 | Accuracy: 9